# Usecase 2: Ocean temperature prediction data preparation

This notebook can be run in the following conda environment:
```shell
mamba env create -f environment_prep_amplicon.yml
conda activate ritme_examples_prep_amplicon
pip install -e .
qiime dev refresh-cache
```

## Setup

In [1]:
import os

import pandas as pd
import qiime2 as q2

from src.seq_fetch_n_process import fetch_mitag_otus
from src.meta_fetch import fetch_mitag_metadata

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
######## USER INPUTS ########
# where to save all the data
destination_folder = "../../data/u2_tara_ocean"
# URL to metadata
url_metadata = "https://ocean-microbiome.embl.de/data/OM.CompanionTables.xlsx"
# URL to otu counts
url_otu_counts = (
    "https://ocean-microbiome.embl.de/data/miTAG.taxonomic.profiles.release.tsv.gz"
)
######## END USER INPUTS #####

## Fetch and process metadata

In [ ]:
fetch_mitag_metadata(destination_folder, url_metadata)

In [ ]:
# get tabs W1: sea basins for stratification from OM.CompanionTables
md_w1 = pd.read_excel(
    "../../data/u2_tara_ocean/OM.CompanionTables.xlsx", sheet_name="Table W1"
)

# rename sample_id column and set as index
sample_id = [x for x in md_w1.columns if x.startswith("Sample label")]
md_w1.rename(columns={sample_id[0]: "sample_id"}, inplace=True)

# rename sea basin column and select
ocean_col = [x for x in md_w1.columns if x.startswith("Ocean and sea regions")]
md_w1.rename(columns={ocean_col[0]: "ocean_basin"}, inplace=True)
md_w1.rename(columns={"PANGAEA sample identifier": "PANGAEA Sample ID"}, inplace=True)
md_selected = md_w1[["sample_id", "PANGAEA Sample ID", "ocean_basin"]].copy()
md_selected.head()

In [ ]:
md_w8 = pd.read_excel(
    "../../data/u2_tara_ocean/OM.CompanionTables.xlsx", sheet_name="Table W8"
)
temp_col = [x for x in md_w8.columns if "temperature" in x.lower()]
md_w8.rename(columns={temp_col[0]: "temperature_mean_degc"}, inplace=True)
md_w8_selected = md_w8[["PANGAEA Sample ID", "temperature_mean_degc"]].copy()

In [ ]:
md_selected.head(2)

In [ ]:
md_w8_selected.head(2)

In [ ]:
# merge
md_merged = pd.merge(md_selected, md_w8_selected, how="inner", on="PANGAEA Sample ID")
md_merged.set_index("sample_id", inplace=True)
# drop column PANGAEA Sample ID
md_merged.drop(columns="PANGAEA Sample ID", inplace=True)
md_merged.head()

## Fetch and process sequences & taxonomy

OTU counts derived from miTAG sequences

In [ ]:
# fetch raw count data
fetch_mitag_otus(destination_folder, url_otu_counts)

In [10]:
mitag_df = pd.read_csv(
    os.path.join(destination_folder, "miTAG.taxonomic.profiles.release.tsv"), sep="\t"
)
mitag_df.rename(columns={"OTU.rep": "Feature ID"}, inplace=True)

In [ ]:
# extract only feature table & save
cols_to_extract = [
    col
    for col in mitag_df.columns
    if col.startswith("Feature ID") or col.startswith("TARA")
]

ft_df = mitag_df[cols_to_extract]
ft_df.set_index("Feature ID", inplace=True)
ft_df = ft_df.T
ft_df.columns.name = None

# save to file
ft_df.to_csv(os.path.join(destination_folder, "otu_table_tara_ocean.tsv"), sep="\t")

ft_df.head()

In [ ]:
# extract taxonomy table & save
cols_for_tax = [
    col
    for col in mitag_df.columns
    if col.startswith("Feature ID") or not col.startswith("TARA")
]
tax_df = mitag_df[cols_for_tax]
tax_df.set_index("Feature ID", inplace=True)

# replace empty space with "_"
tax_cols = ["Domain", "Phylum", "Class", "Order", "Family", "Genus"]
tax_df[tax_cols] = tax_df[tax_cols].apply(lambda col: col.str.replace(" ", "_"))

# compress taxonomy info into "Taxon" column with prefixes
prefixes = ["k__", "p__", "c__", "o__", "f__", "g__"]

tax_df.loc[:, "Taxon"] = tax_df.apply(
    lambda row: "; ".join(
        [
            f"{pre}{row[col]}"
            for pre, col in zip(prefixes, tax_cols)
            if pd.notna(row[col])
        ]
    ),
    axis=1,
)

# save to file
tax_df_to_save = tax_df[["Taxon"]].copy()
tax_art = q2.Artifact.import_data("FeatureData[Taxonomy]", tax_df_to_save)
tax_art.save(os.path.join(destination_folder, "taxonomy_tara_ocean.qza"))

tax_df.head()

## Subset metadata according to feature table

In [ ]:
md_index = md_merged.index.tolist()
ft_df_index = ft_df.index.tolist()

print(md_merged.shape)
md_subset = md_merged.loc[ft_df.index]

path_to_md = os.path.join(destination_folder, "md_tara_ocean.tsv")
md_subset.to_csv(path_to_md, sep="\t")
md_subset.shape